In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir("..")

In [3]:
import torch
from omegaconf import OmegaConf
from train_utils import prepare_data, prepare_model, prepare_optimizer, train, set_seeds

In [4]:
# set seeds
set_seeds()

# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [5]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        image_folder="datasets/train/images/",
        gt_folder="datasets/train/groundtruth/",
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        random_resized_crop_scale=(0.5, 1.0),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=True,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=True,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=True,
        degrees=5,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=True,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=False,  # TODO: Should it always be True?
        # Data loaders
        batch_size=2,
        train_size=0.8,
        val_size=0.1,
        test_size=0.1,
        # Model
        # UNetV1
        model_name="UNetV1",
        model_pretrained=False,
        model_scale=0.5,
        model_save_name="models/checkpoints/unetv1-1.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=0.1,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="unet-v1",
        entity="feeit",
    )
)

In [6]:
# prepare train, validation and test loaders
train_loader, val_loader, test_loader = prepare_data(args)

Using RandomHorizontalFlip.
Using RandomVerticalFlip.
Using RandomRotation with degrees=5.
Using ColorJitter with brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1.
Using ToTensor.


/home/stef/anaconda3/envs/road_segmentation/lib/python3.8/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [7]:
# prepare model
model = prepare_model(args)

Initializing UNetV1 model with pretrained=False, scale=0.5.


Using cache found in /home/stef/.cache/torch/hub/milesial_Pytorch-UNet_master


In [8]:
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()

In [9]:
# prepare optimizer
optimizer = prepare_optimizer(model, args)

Initializing Adam optimizer with lr=0.1.


In [10]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

UNetV1(
  (model): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
            (3): Conv2d(128, 128, kernel_

In [11]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stefan-krsteski (feeit). Use `wandb login --relogin` to force relogin


Best model saved at step:  0
Best model saved at step:  100
Best model saved at step:  300
Best model saved at step:  600
Best model saved at step:  900
Best model saved at step:  1100
Best model saved at step:  1400
Best model saved at step:  1700


Submission

In [12]:
# import transforms
from torchvision import transforms
from datasets.TestDataset import TestDataset
from models.UNetV1 import UNetV1
import matplotlib.pyplot as plt
import torch
from examples.mask_to_submission import *
import torchvision.models.segmentation as models

In [13]:
# select checkpoint
MODEL = "models/checkpoints/unetv1-1.pt"

model = UNetV1()

Using cache found in /home/stef/.cache/torch/hub/milesial_Pytorch-UNet_master


In [14]:
# Load the checkpoint
checkpoint = torch.load(MODEL)

model.load_state_dict(checkpoint)

model.eval()

UNetV1(
  (model): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
            (3): Conv2d(128, 128, kernel_

In [15]:
# path to the test folder
test_folder = "datasets/test/"

# define transformations
transform = transforms.Compose([transforms.ToTensor()])

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

In [16]:
# TODO FIX THIS, bad predictions

In [17]:
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

KeyboardInterrupt: 

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)